# Discretization of data with information gain and entropy

In [679]:
import numpy as np

defining a function to calculate the entropy of a array of with their labels.
It calculate entropy of the labels of the data. you have to pass the labels of the data to this function.

In [680]:
def entropy_calculator(labels):
    uniqe_labels, counts = np.unique(labels, return_counts=True)
    frequency = np.divide(counts, len(labels))

    entropy = -np.sum(frequency * np.log2(frequency))

    return entropy

defining a function that calculates information gain of data based on a splits

In [681]:
def gain_calculator(data, S):
    gain = entropy_calculator(data)

    data = np.array(data)
    # calculate the entropy of each subset
    subset_entropies = np.array([entropy_calculator(subset) for subset in S])
    # calculate the weights of each subset
    weights = np.array([len(subset) / len(data) for subset in S])

    temp = np.sum(weights * subset_entropies)
    
    gain -= temp
    
    return gain

defining a function that splits based of the index of breaking point and the data

In [682]:
def partition(data, breaking_points):
    S = np.split(data, breaking_points)

    return S

## Example

first we get the data and labels, then we sort labels based on the data, then we calculate the entropy of the labels, then we calculate the information gain of the data based on the splits, then we split the data based on the index of the breaking point.

In [683]:
numbers = [1, 2, 3, 4, 5, 6, 7, 8, 9, 10]
labels = [0, 0, 1, 1, 1, 1, 1, 0, 0, 0]

# sort labels based on numbers with numpy

sorted_labels = np.argsort(numbers)
sorted_labels = [labels[i] for i in sorted_labels]

sorted_numbers = np.sort(numbers)

In [684]:
sorted_numbers

array([ 1,  2,  3,  4,  5,  6,  7,  8,  9, 10])

In [685]:
sorted_labels

[0, 0, 1, 1, 1, 1, 1, 0, 0, 0]

partition with 2 break points with indices 2 and 7

In [686]:
S = partition(labels, [2, 7])
S = np.array(S, dtype=object)

S

array([array([0, 0]), array([1, 1, 1, 1, 1]), array([0, 0, 0])],
      dtype=object)

In [687]:
gain_calculator(sorted_labels, S)

1.0

## discretization method 1

just find best breaking point based on the information gain. It wants k which means the number of break points.
It finds best pleace for i_1 among n-1 places, then it finds best pleace for i_2 among n-2 places, then it finds best pleace for i_3 among n-3 places, then it finds best pleace for i_k among n-k places.

fast but not optimal.

O(n^2)

In [688]:
import copy

best_gain = 0
best_breaking_points = []

# just find the best breaking points for each times of k and goes to the next k

def discrete_with_gain_1(data, labels, k):
    global best_gain
    global best_breaking_points

    if k == 0:
        return

    best_bp = 0;
    
    for i in range(1, len(data)):
        if i not in best_breaking_points:


            S = partition(labels, best_breaking_points + [i])
            gain = gain_calculator(labels, S)

            if gain > best_gain:
                best_bp = i
                best_gain = gain
            
    if best_bp != 0:
        best_breaking_points += [best_bp]
        discrete_with_gain_1(data, labels, k-1)
    

In [689]:
discrete_with_gain_1(sorted_numbers, sorted_labels, 2)

In [690]:
best_gain

0.3958156020033583

In [691]:
best_breaking_points

[7]

In [692]:
partition(sorted_labels, best_breaking_points)

[array([0, 0, 1, 1, 1, 1, 1]), array([0, 0, 0])]

## discretization method 2

It recursively checks all possible splits and finds the best split based on the information gain.

It is not efficient but it is more accurate than the first method.

O (n!)

In [693]:
import copy

best_gain = 0
best_breaking_points = []

# calculate all possible combinations of breaking points

def discrete_with_gain_2(data, labels, k, breaking_points = []):
    global best_gain
    global best_breaking_points

    if k == 0:
        return

    best_bp = 0;
    
    for i in range(1, len(data)):
        if i not in breaking_points:

            breaking_points += [i]
            S = partition(labels, breaking_points)
            gain = gain_calculator(labels, S)

            if gain > best_gain:
                best_bp = i
                best_gain = gain
                best_breaking_points = copy.deepcopy(breaking_points)

            discrete_with_gain_2(data, labels, k-1, breaking_points)
            breaking_points.pop()
            
    

In [694]:
discrete_with_gain_2(sorted_numbers, sorted_labels, 2)

In [695]:
best_gain, best_breaking_points

(1.0, [2, 7])

In [696]:
partition(sorted_labels, best_breaking_points)

[array([0, 0]), array([1, 1, 1, 1, 1]), array([0, 0, 0])]

## Testing on Titanic dataset, for discretization of age

* note that you just can discretize the numerical data, not the categorical data.

In [697]:
import pandas as pd

df = pd.read_csv('train.csv')

df.head()

,PassengerId,Survived,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked
0,1,0,3,"Braund, Mr. Owen Harris",male,22.0,1,0,A/5 21171,7.2500,NaN,S
1,2,1,1,"Cumings, Mrs. John Bradley (Florence Briggs Th...",female,38.0,1,0,PC 17599,71.2833,C85,C
2,3,1,3,"Heikkinen, Miss. Laina",female,26.0,0,0,STON/O2. 3101282,7.9250,NaN,S
3,4,1,1,"Futrelle, Mrs. Jacques Heath (Lily May Peel)",female,35.0,1,0,113803,53.1000,C123,S
4,5,0,3,"Allen, Mr. William Henry",male,35.0,0,0,373450,8.0500,NaN,S


In [698]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 891 entries, 0 to 890
Data columns (total 12 columns):
 #   Column       Non-Null Count  Dtype  
---  ------       --------------  -----  
 0   PassengerId  891 non-null    int64  
 1   Survived     891 non-null    int64  
 2   Pclass       891 non-null    int64  
 3   Name         891 non-null    object 
 4   Sex          891 non-null    object 
 5   Age          714 non-null    float64
 6   SibSp        891 non-null    int64  
 7   Parch        891 non-null    int64  
 8   Ticket       891 non-null    object 
 9   Fare         891 non-null    float64
 10  Cabin        204 non-null    object 
 11  Embarked     889 non-null    object 
dtypes: float64(2), int64(5), object(5)
memory usage: 83.7+ KB


In [699]:
df = df[['Survived', 'Age']]
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 891 entries, 0 to 890
Data columns (total 2 columns):
 #   Column    Non-Null Count  Dtype  
---  ------    --------------  -----  
 0   Survived  891 non-null    int64  
 1   Age       714 non-null    float64
dtypes: float64(1), int64(1)
memory usage: 14.0 KB


In [700]:
df = df.dropna()
df.info()

<class 'pandas.core.frame.DataFrame'>
Index: 714 entries, 0 to 890
Data columns (total 2 columns):
 #   Column    Non-Null Count  Dtype  
---  ------    --------------  -----  
 0   Survived  714 non-null    int64  
 1   Age       714 non-null    float64
dtypes: float64(1), int64(1)
memory usage: 16.7 KB


In [701]:
df.describe()

,Survived,Age
count,714.000000,714.000000
mean,0.406162,29.699118
std,0.491460,14.526497
min,0.000000,0.420000
25%,0.000000,20.125000
50%,0.000000,28.000000
75%,1.000000,38.000000
max,1.000000,80.000000


In [702]:
labels = df['Survived'].values
data = df['Age'].values

In [703]:
sorted_labels = np.argsort(data)
sorted_labels = [labels[i] for i in sorted_labels]

sorted_data = np.sort(data)

In [704]:
sorted_data

array([ 0.42,  0.67,  0.75,  0.75,  0.83,  0.83,  0.92,  1.  ,  1.  ,
        1.  ,  1.  ,  1.  ,  1.  ,  1.  ,  2.  ,  2.  ,  2.  ,  2.  ,
        2.  ,  2.  ,  2.  ,  2.  ,  2.  ,  2.  ,  3.  ,  3.  ,  3.  ,
        3.  ,  3.  ,  3.  ,  4.  ,  4.  ,  4.  ,  4.  ,  4.  ,  4.  ,
        4.  ,  4.  ,  4.  ,  4.  ,  5.  ,  5.  ,  5.  ,  5.  ,  6.  ,
        6.  ,  6.  ,  7.  ,  7.  ,  7.  ,  8.  ,  8.  ,  8.  ,  8.  ,
        9.  ,  9.  ,  9.  ,  9.  ,  9.  ,  9.  ,  9.  ,  9.  , 10.  ,
       10.  , 11.  , 11.  , 11.  , 11.  , 12.  , 13.  , 13.  , 14.  ,
       14.  , 14.  , 14.  , 14.  , 14.  , 14.5 , 15.  , 15.  , 15.  ,
       15.  , 15.  , 16.  , 16.  , 16.  , 16.  , 16.  , 16.  , 16.  ,
       16.  , 16.  , 16.  , 16.  , 16.  , 16.  , 16.  , 16.  , 16.  ,
       16.  , 17.  , 17.  , 17.  , 17.  , 17.  , 17.  , 17.  , 17.  ,
       17.  , 17.  , 17.  , 17.  , 17.  , 18.  , 18.  , 18.  , 18.  ,
       18.  , 18.  , 18.  , 18.  , 18.  , 18.  , 18.  , 18.  , 18.  ,
       18.  , 18.  ,

In [705]:
sorted_labels

[1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 0,
 1,
 0,
 1,
 1,
 0,
 0,
 1,
 0,
 0,
 0,
 0,
 1,
 1,
 0,
 1,
 1,
 0,
 1,
 1,
 1,
 0,
 0,
 0,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 0,
 1,
 0,
 1,
 0,
 0,
 1,
 1,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 1,
 1,
 0,
 0,
 0,
 0,
 0,
 1,
 1,
 1,
 1,
 1,
 0,
 1,
 1,
 0,
 0,
 0,
 0,
 1,
 1,
 1,
 1,
 1,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 1,
 0,
 1,
 0,
 1,
 0,
 1,
 1,
 0,
 0,
 0,
 1,
 1,
 0,
 0,
 0,
 1,
 1,
 1,
 1,
 0,
 0,
 0,
 1,
 0,
 0,
 0,
 0,
 0,
 1,
 1,
 0,
 0,
 1,
 1,
 1,
 0,
 1,
 0,
 1,
 1,
 0,
 0,
 0,
 0,
 0,
 0,
 1,
 1,
 0,
 0,
 1,
 0,
 1,
 0,
 1,
 0,
 0,
 0,
 1,
 0,
 0,
 0,
 0,
 1,
 1,
 0,
 0,
 0,
 0,
 0,
 1,
 0,
 1,
 1,
 0,
 0,
 0,
 1,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 1,
 1,
 0,
 0,
 0,
 0,
 0,
 1,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 1,
 0,
 0,
 0,
 1,
 0,
 0,
 1,
 1,
 0,
 0,
 0,
 1,
 0,
 0,
 1,
 1,
 0,
 1,
 0,
 0,
 1,
 0,
 1,
 0,
 0,
 1,
 1,
 1,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 1,
 0,
 0,
 0,
 1,
 0,
 0,
 0,
 1,
 0,
 1,
 1,
 0,
 1,
 0,
 0,


In [706]:
best_gain = 0
best_breaking_points = []

discrete_with_gain_1(sorted_data, sorted_labels, 2)

best_gain, best_breaking_points

(0.02519115408620176, [45, 701])

In [707]:
sorted_data[0], sorted_data[best_breaking_points[0]], sorted_data[best_breaking_points[1]], sorted_data[-1]

(0.42, 6.0, 64.0, 80.0)

### Discretization method 3 based on Genetic Algorithm

In [708]:
class Genetic_algorithm:

    def __init__(self, number_of_generations, population_size, number_of_genes, mutation_percent, fitness_func, gene_range):
        self.number_of_generations = number_of_generations
        self.population_size = population_size
        self.number_of_genes = number_of_genes
        self.mutation_percent = mutation_percent
        self.fitness_func = fitness_function
        self.best_solution = None
        self.gene_range = gene_range

        self.chromosomes = np.zeros((self.population_size+self.number_of_generations, self.number_of_genes), dtype=int)
        self.fitness = np.zeros((self.population_size + self.number_of_generations+1,1))

    def population(self):
        indicies = range(self.gene_range[0], self.gene_range[1])
        for i in range(self.population_size):
            new_chromosome = np.random.choice(indicies, self.number_of_genes, replace=False)

            self.chromosomes[i] = new_chromosome
            self.fitness[i] = self.fitness_func(new_chromosome)

    def selection(self):
        # get index of fittest and second fittest chromosomes

        fittest_index = np.argmax(self.fitness)
        second_fittest_index = np.argmax(np.delete(self.fitness, fittest_index, axis=0))

        # Store the fittest and second fittest chromosomes
        parent_1 = self.chromosomes[fittest_index]
        parent_2 = self.chromosomes[second_fittest_index]

        # get random chromosomes and store it in parent_3

        parent_3 = self.chromosomes[np.random.randint(0, self.population_size)]

        return parent_1, parent_2, parent_3


    def crossover(self, parent_1, parent_2, parent_3):
        # create new chromosome by crossover between parent_1, parent_2 and parent_3
        # randomly select genes from parent_1, parent_2 and parent_3
        new_chromosome = []
        for i in range(self.number_of_genes):
            chance = np.random.randint(0, 100)
            if chance <= 50: 
                new_chromosome.append(parent_1[i])
            elif chance <= 88:
                new_chromosome.append(parent_2[i])
            else:
                new_chromosome.append(parent_3[i])


        return new_chromosome

    def mutation(self, new_chromosome):
        chance = np.random.randint(0, 100)

        for i in range(self.number_of_genes):
            chance = np.random.randint(0, 100)

            if chance <= self.mutation_percent:
                new_chromosome[i] = np.random.choice(self.gene_range)

        return new_chromosome

    
    def termination(self):
        min_index = np.argmin(self.fitness)
        np.delete(self.fitness, min_index)
        np.delete(self.chromosomes, min_index)


    def best_solution(self):
        return self.best_solution

    def run(self):
        self.population()
        for i in range(self.population_size, self.population_size + self.number_of_generations):
            parent_1, parent_2, parent_3 = self.selection()
            new_chromosome = self.crossover(parent_1, parent_2, parent_3)
            self.termination()
            if np.random.randint(0, 100) <= self.mutation_percent:
                new_chromosome = self.mutation(new_chromosome)
            new_chromosome_fitness = self.fitness_func(new_chromosome)
            self.chromosomes[i]=new_chromosome
            self.fitness[i]=new_chromosome_fitness
        self.best_solution = self.chromosomes[np.argmax(self.fitness)]

In [709]:
def fitness_function(chromosome):
    S = partition(sorted_labels, chromosome)
    return gain_calculator(sorted_labels, S)

In [710]:

ga = Genetic_algorithm(
    number_of_generations=2000,
    population_size=50,
    number_of_genes=2,
    mutation_percent=35,
    fitness_func=fitness_function,
    gene_range=(0, len(sorted_data))
)

In [711]:
ga.run()

In [712]:
ga.best_solution

array([ 9, 52])

In [713]:
gain_calculator(sorted_labels, partition(sorted_labels, ga.best_solution))

0.02453973093470152

In [714]:
best_gain = 0
best_breaking_points = []

discrete_with_gain_2(sorted_data, sorted_labels, 2)

best_gain, best_breaking_points

(0.02842567469977031, [33, 45])

In [715]:
sorted_data[0], sorted_data[best_breaking_points[0]], sorted_data[best_breaking_points[1]], sorted_data[-1]

(0.42, 4.0, 6.0, 80.0)